In [1]:
print('Takumi House & Kailen Santos')

Takumi House & Kailen Santos


In [2]:
from matplotlib import pyplot as plt
import matplotlib as mp
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import sklearn.manifold
from scipy import cluster
import seaborn as sns
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
import requests
import json
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import re
import json
import pickle
%matplotlib inline

In [3]:
try:
    with open('df_reviews','rb') as f:
        df_reviews = pickle.load(f)
        
except FileNotFoundError:
    with open('df_reviews','wb') as f:
        pickle.dump(df_reviews, f)

In [4]:
try:
    with open('stemmed_data','rb') as f:
        stemmed_data = pickle.load(f)
        
except FileNotFoundError:
    stemmed_data = [" ".join(SnowballStemmer("english", ignore_stopwords=True).stem(word)  
        for sent in sent_tokenize(message)
        for word in word_tokenize(sent))
        for message in df_reviews['text']]
    
    with open('stemmed_data','wb') as f:
        pickle.dump(stemmed_data, f)

In [5]:
df = pd.read_csv('Restaurant_Inspections.csv',header=0)
# df = df[df.City == 'Las Vegas']
# df = (df[df['Inspection Result'] == 'Closed with Fees']).append(df[df['Inspection Result'] == 'Closed without Fees'])

df.shape

#with open ('Resaurant_Inspections.csv') as file:
#   for line in file:
#      if (line[6]=='Las Vegas' && line[]

C:\Users\Kailen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(123919, 23)

In [6]:
df = df[df.City == 'Las Vegas']
df.shape

(94700, 23)

In [7]:
categories = df['Category Name'].unique().tolist()
print(categories)

['Pantry', 'Special Kitchen', 'Garde Manger', 'Kitchen Bakery', 'Bar / Tavern', 'Buffet', 'Snack Bar', 'Restaurant', 'Banquet Support', 'Portable Unit', 'Banquet Kitchen', 'Portable Bar', 'Meat/Poultry/Seafood', 'Vegetable Prep', 'Caterer', 'Barbeque', 'Confection', 'Concessions', 'Produce Market', 'Institutional Food Service', 'Elementary School Kitchen', 'Self-Service Food Truck', 'Bakery Sales', 'Food Trucks / Mobile Vendor', 'Grocery Store Sampling', 'Beer Bar', 'Childcare Kitchens', "Farmer's Market", 'Main Kitchen', 'Frozen Meat Sales']


In [8]:
categories.remove('Elementary School Kitchen')
categories.remove('Grocery Store Sampling')
categories.remove('Farmer\'s Market')
categories.remove('Caterer')
categories.remove('Concessions')
categories.remove('Institutional Food Service')
categories.remove('Banquet Support')

In [9]:
df = df[df['Category Name'].isin(categories)]
df.shape

(91735, 23)

In [10]:
coord_strings = df['Location 1'].tolist()

In [11]:
import ast
xcoords = []
ycoords = []
for coord in coord_strings:
    xcoords.append(ast.literal_eval(coord)[0])
    ycoords.append(ast.literal_eval(coord)[1])

In [12]:
count = 0
for idx, ycoord in enumerate(ycoords):
    if ycoord > 0:
        ycoords[idx] = ycoord * -1
        count += 1
print('{} coordinates fixed.'.format(count))

574 coordinates fixed.


In [13]:
df['Coordinates'] = list(zip(xcoords, ycoords))
df.head()

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Employee ID,Inspection Type,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates
11,DA0001715,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
12,DA0001716,PR0000426,CAESARS AUGUSTUS BUTLER PANTRY,CAESARS PALACE HOTEL & CASINO,Pantry,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000567,Routine Inspection,2,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
13,DA0001728,PR0000427,CAESARS DIAMOND LOUNGE KITCHEN,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,8,...,EE7000417,Routine Inspection,4,A,NaN,Compliant,14,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
14,DA0001773,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000417,Routine Inspection,7,A,NaN,Compliant,14232831,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"
15,DA0001776,PR0000434,CAESARS BAKERY WAREWASH,CAESARS PALACE HOTEL & CASINO,Special Kitchen,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,3,...,EE7000392,Routine Inspection,2,A,NaN,Compliant,NaN,01/22/2014 02:03:44 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)"


In [14]:
coordinates_list = df['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])
df['X'] = xcoords
df['Y'] = ycoords

In [15]:
df = df[df['Y'] < -114.5]
df = df[df['Y'] > -115.6]
df_clean = df.reset_index(drop = True)

In [16]:
coordinates_list = df_clean['Coordinates'].tolist()
xcoords = []
ycoords = []
for coordinates in coordinates_list:
    xcoords.append(coordinates[0])
    ycoords.append(coordinates[1])

In [17]:
df = df_clean[df_clean['Date Current']==df_clean['Inspection Date']]
df

,Serial Number,Permit Number,Restaurant Name,Location Name,Category Name,Address,City,State,Zip,Current Demerits,...,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,Record Updated,Location 1,Coordinates,X,Y
63,DA0003413,PR0000492,CAESARS SEASONAL VENUS POOL BAR DELETED,CAESARS PALACE HOTEL & CASINO,Portable Unit,3570 S Las Vegas Blvd,Las Vegas,Nevada,89109-8924,4,...,4,A,NaN,Compliant,114,02/21/2013 10:26:12 PM,"(36.1161559, -115.1750576)","(36.1161559, -115.1750576)",36.116156,-115.175058
196,DA0009343,PR0000728,MIRAGE 1 OAK SMALL BAR 2 ORIGINAL DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,10,...,10,A,NaN,Compliant,64114,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
254,DA0011310,PR0000730,MIRAGE 1 OAK MULTILEVEL BAR 4 DELETED,MIRAGE HOTEL & CASINO,Bar / Tavern,3400 S Las Vegas Blvd,Las Vegas,Nevada,89109-8923,3,...,3,A,NaN,Compliant,13,02/21/2013 10:26:12 PM,"(36.1206015, -115.1768382)","(36.1206015, -115.1768382)",36.120601,-115.176838
317,DA0017310,PR0000631,Golden Gate Beverage Cart,Golden Gate Hotel & Casino,Portable Unit,1 Fremont St,Las Vegas,Nevada,89101-5601,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1713005, -115.1462312)","(36.1713005, -115.1462312)",36.171301,-115.146231
348,DA001XLIF,PR0022088,Marketon #1 - Deli,Marketon #1,Snack Bar,840 N Decatur Blvd,Las Vegas,Nevada,89107-1905,8,...,8,A,NaN,'A' Grade,209217,07/11/2016 03:47:09 PM,"(36.180332, -115.205031)","(36.180332, -115.205031)",36.180332,-115.205031
436,DA0023482,PR0001171,Aramark Nacho Cart,ARAMARK @ CASHMAN CENTER,Portable Unit,850 N LAS VEGAS Blvd,Las Vegas,Nevada,89101,0,...,0,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1800444, -115.131106)","(36.1800444, -115.131106)",36.180044,-115.131106
503,DA0025038,PR0001066,Kentucky Fried Chicken #527621,Kentucky Fried Chicken #X527621,Restaurant,4065 S Buffalo Dr,Las Vegas,Nevada,89147-7455,7,...,7,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.1150528, -115.2625292)","(36.1150528, -115.2625292)",36.115053,-115.262529
505,DA0025516,PR0001062,Kentucky Fried Chicken / Pizza Hut,Kentucky Fried Chicken/Pizza Hut #X527066,Restaurant,5587 S Rainbow Blvd,Las Vegas,Nevada,89118-1857,5,...,5,A,NaN,Compliant,1420,02/21/2013 10:26:12 PM,"(36.0877539, -115.243508)","(36.0877539, -115.243508)",36.087754,-115.243508
531,DA0027055,PR0001064,Kentucky Fried Chicken #X527030,Kentucky Fried Chicken,Restaurant,9360 W Sahara Ave,Las Vegas,Nevada,89117-5351,2,...,2,A,NaN,Compliant,NaN,02/21/2013 10:26:12 PM,"(36.144667, -115.299057)","(36.144667, -115.299057)",36.144667,-115.299057
533,DA0027285,PR0001068,Kentucky Fried Chicken #527019,Kentucky Fried Chicken #X527019,Restaurant,2490 E Desert Inn Rd,Las Vegas,Nevada,89121-3617,10,...,10,A,NaN,Compliant,1314203136,02/21/2013 10:26:12 PM,"(36.1301516, -115.116572)","(36.1301516, -115.116572)",36.130152,-115.116572


In [18]:
df_clean = df.reset_index(drop=True)

In [19]:
df_agg = df_clean.groupby('Restaurant Name',as_index=False).agg(lambda x: x.tolist())

In [20]:
dfcategories = df_agg.columns
dfcategories

Index(['Restaurant Name', 'Serial Number', 'Permit Number', 'Location Name',
       'Category Name', 'Address', 'City', 'State', 'Zip', 'Current Demerits',
       'Current Grade', 'Date Current', 'Inspection Date', 'Inspection Time',
       'Employee ID', 'Inspection Type', 'Inspection Demerits',
       'Inspection Grade', 'Permit Status', 'Inspection Result', 'Violations',
       'Record Updated', 'Location 1', 'Coordinates', 'X', 'Y'],
      dtype='object')

In [21]:
# drop unnecessary columns for categorization step later
df_agg = df_agg.drop(['Serial Number'], axis = 1)
df_agg = df_agg.drop(['City'], axis = 1)
df_agg = df_agg.drop(['State'], axis = 1)
df_agg = df_agg.drop(['Zip'], axis = 1)
df_agg = df_agg.drop(['Inspection Time'], axis = 1)
df_agg = df_agg.drop(['Employee ID'], axis = 1)
df_agg = df_agg.drop(['Inspection Type'], axis = 1)
df_agg = df_agg.drop(['Location 1'], axis = 1)
df_agg = df_agg.drop(['Coordinates'], axis = 1)
df_agg = df_agg.drop(['Record Updated'], axis = 1)
df_agg = df_agg.drop(['Inspection Date'], axis = 1)

In [22]:
df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,#1 HAWAIIAN BARBEQUE,[PR0110047],[CHINA DRAGON/#1 HAWAIIAN BARBEQUE],[Restaurant],[5905 S EASTERN Ave 105],[9],[A],[12/21/2015 12:00:00 AM],[9],[A],[nan],[Compliant],[211215292829302955],[36.0820114],[-115.119644]
1,$5 CAFE,[PR0017837],[AZTEC INN - $5 CAFE],[Restaurant],[2200 S Las Vegas Blvd],[10],[A],[01/07/2016 12:00:00 AM],[10],[A],[nan],[Compliant],[202206291029282930],[36.1459136],[-115.1561175]
2,100 DEGREES HOT POT - DELETED 10/15,[PR0113718],[100 DEGREES HOT POT],[Restaurant],[5600 SPRING MOUNTAIN Rd B],[3],[A],[04/27/2015 12:00:00 AM],[3],[A],[nan],[Compliant],[2122911],[36.1265084],[-115.2182851]
3,101 MART SNACK BAR,[PR0118514],[101 MART],[Snack Bar],[1021 CONVENTION CENTER Dr],[6],[A],[07/26/2016 12:00:00 AM],[6],[A],[nan],['A' Grade],[2152955],[36.1322288],[-115.1553506]
4,168 Market - Bakery,[PR0021447],[168 Market],[Bakery Sales],[3459 S Jones Blvd],[3],[A],[07/05/2016 12:00:00 AM],[3],[A],[nan],['A' Grade],[217],[36.1265813],[-115.2250088]
5,168 Market - Restaurant,[PR0021446],[168 Market],[Restaurant],[3459 S Jones Blvd],[0],[A],[10/14/2015 12:00:00 AM],[0],[A],[nan],[Compliant],[291029282930],[36.1265813],[-115.2250088]
6,168 Market / Produce Department,[PR0021445],[168 Market],[Produce Market],[3459 S Jones Blvd],[6],[A],[07/28/2016 12:00:00 AM],[6],[A],[nan],['A' Grade],[2152955],[36.1265813],[-115.2250088]
7,17 SOUTH BOOZE & BITES BAR,[PR0109450],[Tahiti Village],[Bar / Tavern],[7200 S Las Vegas Blvd],[0],[A],[05/18/2016 12:00:00 AM],[0],[A],[nan],[Compliant],[nan],[36.0596893],[-115.1715274]
8,17 SOUTH BOOZE & BITES GRILL,[PR0109449],[Tahiti Village],[Restaurant],[7200 S Las Vegas Blvd],[9],[A],[05/18/2016 12:00:00 AM],[9],[A],[nan],[Compliant],[21221629292955],[36.0596893],[-115.1715274]
9,1900 ASIAN CUISINE,[PR0099062],[1900 ASIAN CUISINE],[Restaurant],[5115 Spring Mountain Rd 103],[0],[A],[11/22/2013 12:00:00 AM],[0],[A],[nan],[Compliant],[29102927],[36.1255374],[-115.2112409]


In [23]:
dfcategories = df_agg.columns
for x in dfcategories[1:]:
    for y in range(len(df_agg[x])):
        df_agg[x][y] = list(set(df_agg[x][y]))
        df_agg[x][y] = df_agg[x][y][0]

In [24]:
df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Inspection Demerits,Inspection Grade,Permit Status,Inspection Result,Violations,X,Y
0,#1 HAWAIIAN BARBEQUE,PR0110047,CHINA DRAGON/#1 HAWAIIAN BARBEQUE,Restaurant,5905 S EASTERN Ave 105,9,A,12/21/2015 12:00:00 AM,9,A,NaN,Compliant,211215292829302955,36.082,-115.12
1,$5 CAFE,PR0017837,AZTEC INN - $5 CAFE,Restaurant,2200 S Las Vegas Blvd,10,A,01/07/2016 12:00:00 AM,10,A,NaN,Compliant,202206291029282930,36.1459,-115.156
2,100 DEGREES HOT POT - DELETED 10/15,PR0113718,100 DEGREES HOT POT,Restaurant,5600 SPRING MOUNTAIN Rd B,3,A,04/27/2015 12:00:00 AM,3,A,NaN,Compliant,2122911,36.1265,-115.218
3,101 MART SNACK BAR,PR0118514,101 MART,Snack Bar,1021 CONVENTION CENTER Dr,6,A,07/26/2016 12:00:00 AM,6,A,NaN,'A' Grade,2152955,36.1322,-115.155
4,168 Market - Bakery,PR0021447,168 Market,Bakery Sales,3459 S Jones Blvd,3,A,07/05/2016 12:00:00 AM,3,A,NaN,'A' Grade,217,36.1266,-115.225
5,168 Market - Restaurant,PR0021446,168 Market,Restaurant,3459 S Jones Blvd,0,A,10/14/2015 12:00:00 AM,0,A,NaN,Compliant,291029282930,36.1266,-115.225
6,168 Market / Produce Department,PR0021445,168 Market,Produce Market,3459 S Jones Blvd,6,A,07/28/2016 12:00:00 AM,6,A,NaN,'A' Grade,2152955,36.1266,-115.225
7,17 SOUTH BOOZE & BITES BAR,PR0109450,Tahiti Village,Bar / Tavern,7200 S Las Vegas Blvd,0,A,05/18/2016 12:00:00 AM,0,A,NaN,Compliant,NaN,36.0597,-115.172
8,17 SOUTH BOOZE & BITES GRILL,PR0109449,Tahiti Village,Restaurant,7200 S Las Vegas Blvd,9,A,05/18/2016 12:00:00 AM,9,A,NaN,Compliant,21221629292955,36.0597,-115.172
9,1900 ASIAN CUISINE,PR0099062,1900 ASIAN CUISINE,Restaurant,5115 Spring Mountain Rd 103,0,A,11/22/2013 12:00:00 AM,0,A,NaN,Compliant,29102927,36.1255,-115.211


In [25]:
try:
    with open('non_0_updated_sick_scores','rb') as f:
        non_0_updated_sick_scores = pickle.load(f)
        
except FileNotFoundError:
    with open('non_0_updated_sick_scores','wb') as f:
        pickle.dump(non_0_sick_scores, f)

In [26]:
file = 'yelp_academic_dataset_business.json'
business_locations = {}
lv_restaurant_ids = []
with open(file) as f:
    for line in f:
        r_line = json.loads(line)
        business_locations.update({r_line['business_id']:r_line['full_address']})
        if 'Restaurants' in r_line['categories']:
            lv_restaurant_ids.append(r_line['business_id'])

In [27]:
sick_ids = []
for x, y in non_0_updated_sick_scores.items():
    sick_ids.append(x)
len(sick_ids)

1942

In [28]:
addresses = []
for x, y in business_locations.items():
    if x in sick_ids:
        addresses.append(y.rsplit('\n', 5)[0].lower())
len(addresses)

1942

In [29]:
addresses

['south point hotel & casino',
 '1916 village center cir',
 'renaissance las vegas',
 '8410 w cheyenne ave',
 '10690 southern highlands pkwy',
 'palms casino resort',
 'mgm grand',
 '410 s rampart blvd',
 '2620 regatta dr',
 "harrah's las vegas hotel & casino",
 '1383 e silverado ranch blvd',
 '2345 e centennial pkwy',
 'fashion show mall',
 '5115 w spring mountain rd',
 '1031 s rampart blvd',
 'rio hotel and casino',
 '2101 n rainbow blvd',
 '1260 n town center dr',
 'fashion show mall',
 '6115 s fort apache rd',
 '7620 las vegas boulevard',
 '8975 s eastern ave',
 'planet hollywood resort & casino',
 'mandalay bay resort and casino',
 '2025 village center cir',
 'treasure island',
 'rio all-suite hotel & casino',
 '4670 s decatur blvd',
 'aria resort & casino',
 '6630 n durango dr',
 '9410 w sahara ave',
 '4029 spring mountain rd',
 '4355 spring mountain rd',
 'new york new york hotel & casino',
 '4711 spring mountain rd',
 '7777 w sahara ave',
 '3230 e flamingo rd',
 '520 fremont st

In [30]:
# create lists for addresses of businesses with violations in order to 
# check for overlap between the Las Vegas Violation set and the yelp set.

violation_addresses = []
overlap = []

for line in df_agg['Address']:
    violation_addresses.append(str(line).lower())
violation_addresses[:10]

for line in addresses:
    if line in violation_addresses:
        overlap.append(line)
print(addresses[:20])
print(overlap[:20])

['south point hotel & casino', '1916 village center cir', 'renaissance las vegas', '8410 w cheyenne ave', '10690 southern highlands pkwy', 'palms casino resort', 'mgm grand', '410 s rampart blvd', '2620 regatta dr', "harrah's las vegas hotel & casino", '1383 e silverado ranch blvd', '2345 e centennial pkwy', 'fashion show mall', '5115 w spring mountain rd', '1031 s rampart blvd', 'rio hotel and casino', '2101 n rainbow blvd', '1260 n town center dr', 'fashion show mall', '6115 s fort apache rd']
['8410 w cheyenne ave', '10690 southern highlands pkwy', '2620 regatta dr', '1383 e silverado ranch blvd', '1031 s rampart blvd', '2101 n rainbow blvd', '1260 n town center dr', '6115 s fort apache rd', '8975 s eastern ave', '2025 village center cir', '4670 s decatur blvd', '6630 n durango dr', '9410 w sahara ave', '4355 spring mountain rd', '4711 spring mountain rd', '7777 w sahara ave', '520 fremont st', '3055 e flamingo rd', '8540 w lake mead blvd', '7435 s eastern ave']


In [31]:
print(len(overlap))

911


It looks like there are a total of 911 unique restaurant addresses that have an evaluated sick score greater than 0 and entries in the inspection dataset.

In [32]:
categories = df_agg.columns
df_agg = df_agg.drop(['Inspection Grade'], axis=1)
df_agg = df_agg.drop(['Inspection Demerits'], axis=1)              

In [33]:
df_agg

,Restaurant Name,Permit Number,Location Name,Category Name,Address,Current Demerits,Current Grade,Date Current,Permit Status,Inspection Result,Violations,X,Y
0,#1 HAWAIIAN BARBEQUE,PR0110047,CHINA DRAGON/#1 HAWAIIAN BARBEQUE,Restaurant,5905 S EASTERN Ave 105,9,A,12/21/2015 12:00:00 AM,NaN,Compliant,211215292829302955,36.082,-115.12
1,$5 CAFE,PR0017837,AZTEC INN - $5 CAFE,Restaurant,2200 S Las Vegas Blvd,10,A,01/07/2016 12:00:00 AM,NaN,Compliant,202206291029282930,36.1459,-115.156
2,100 DEGREES HOT POT - DELETED 10/15,PR0113718,100 DEGREES HOT POT,Restaurant,5600 SPRING MOUNTAIN Rd B,3,A,04/27/2015 12:00:00 AM,NaN,Compliant,2122911,36.1265,-115.218
3,101 MART SNACK BAR,PR0118514,101 MART,Snack Bar,1021 CONVENTION CENTER Dr,6,A,07/26/2016 12:00:00 AM,NaN,'A' Grade,2152955,36.1322,-115.155
4,168 Market - Bakery,PR0021447,168 Market,Bakery Sales,3459 S Jones Blvd,3,A,07/05/2016 12:00:00 AM,NaN,'A' Grade,217,36.1266,-115.225
5,168 Market - Restaurant,PR0021446,168 Market,Restaurant,3459 S Jones Blvd,0,A,10/14/2015 12:00:00 AM,NaN,Compliant,291029282930,36.1266,-115.225
6,168 Market / Produce Department,PR0021445,168 Market,Produce Market,3459 S Jones Blvd,6,A,07/28/2016 12:00:00 AM,NaN,'A' Grade,2152955,36.1266,-115.225
7,17 SOUTH BOOZE & BITES BAR,PR0109450,Tahiti Village,Bar / Tavern,7200 S Las Vegas Blvd,0,A,05/18/2016 12:00:00 AM,NaN,Compliant,NaN,36.0597,-115.172
8,17 SOUTH BOOZE & BITES GRILL,PR0109449,Tahiti Village,Restaurant,7200 S Las Vegas Blvd,9,A,05/18/2016 12:00:00 AM,NaN,Compliant,21221629292955,36.0597,-115.172
9,1900 ASIAN CUISINE,PR0099062,1900 ASIAN CUISINE,Restaurant,5115 Spring Mountain Rd 103,0,A,11/22/2013 12:00:00 AM,NaN,Compliant,29102927,36.1255,-115.211


In [58]:
df_cat = pd.get_dummies(df_agg['Category Name'], prefix = 'category')
df_grade = pd.get_dummies(df_agg['Current Grade'], prefix = 'grade_cur')
df_perm = pd.get_dummies(df_agg['Permit Status'], prefix = 'permit')
df_res = pd.get_dummies(df_agg['Inspection Result'], prefix = 'result')
df_dems = pd.get_dummies(df_agg['Current Demerits'], prefix = 'demerits')
df_new = pd.concat([df_agg, df_cat], axis=1)
df_new = pd.concat([df_new, df_grade], axis=1)
df_new = pd.concat([df_new, df_perm], axis=1)
df_new = pd.concat([df_new, df_res], axis=1)
df_new = pd.concat([df_new, df_dems], axis=1)
df_new = df_new.drop(['Category Name'], axis = 1 )
df_new = df_new.drop(['Current Grade'], axis = 1 )
df_new = df_new.drop(['Permit Status'], axis = 1 )
df_new = df_new.drop(['Inspection Result'], axis = 1)
df_new = df_new.drop(['Date Current'], axis = 1)
df_new = df_new.drop(['Restaurant Name'], axis = 1)
df_new = df_new.drop(['Permit Number'], axis = 1)
df_new = df_new.drop(['Location Name'], axis = 1)
df_new = df_new.drop(['Address'], axis = 1)
df_new = df_new.drop(['Violations'], axis = 1)

In [59]:
df_new

,Current Demerits,X,Y,category_Bakery Sales,category_Banquet Kitchen,category_Bar / Tavern,category_Barbeque,category_Beer Bar,category_Buffet,category_Childcare Kitchens,...,demerits_36,demerits_37,demerits_38,demerits_39,demerits_42,demerits_43,demerits_46,demerits_51,demerits_53,demerits_61
0,9,36.082,-115.12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10,36.1459,-115.156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,36.1265,-115.218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,36.1322,-115.155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3,36.1266,-115.225,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,36.1266,-115.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,36.1266,-115.225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0,36.0597,-115.172,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,36.0597,-115.172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0,36.1255,-115.211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

X = df_new.loc[:,:].values
y = df_new['grade_cur_X'].values
X_train, X_test, y_train, y_test = train_test_split(X, y)
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)
print(pred)
test_score = knn.score(X_test,y_test)
print('Accuracy on test data: {}'.format(test_score))


[ 0.  0.  0. ...,  0.  0.  0.]
Accuracy on test data: 0.9997120644975526


In [62]:
X = df_new.loc[:,:].values
y = df_new['grade_cur_A'].values
X_train, X_test, y_train, y_test = train_test_split(X, y)
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)
print(pred)
test_score = knn.score(X_test,y_test)
print('Accuracy on test data: {}'.format(test_score))

[ 1.  1.  1. ...,  1.  1.  1.]
Accuracy on test data: 0.9979844514828679
